# Step 4 - Training the Model

In [Step 2](Step 2 - Choosing a Model.ipynb), we determined we'll use **InceptionV3** as our model.
In [Step 3](Step 3 - Fine-tuning the Model.ipynb), we have decided:
- We'll use one hidden layer of **256** nodes 
- A dropout layer of **0.4**.

Now that we have a good understanding of our model, it's time to train the model.

In [1]:
# import dependencies
from image_preprocessor import ImagePreprocessor

import numpy as np
import random
import os
import shutil
from glob import glob

from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.preprocessing import image as Image

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

import keras.callbacks as callbacks
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model
from keras.callbacks import TensorBoard

import re
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Using TensorFlow backend.


In [2]:
root_dir = os.path.join('..', 'application', 'images')
originals_dir = os.path.join(root_dir, "original")
training_dir = os.path.join(root_dir, "train")
test_dir = os.path.join(root_dir, "test")
val_dir = os.path.join(root_dir, "validation")

target_imagesize = (256, 256)

clear_existing_data = False # if true, data in training, test and validation directories will be deleted before splitting the data in the originals directory
augment_data = True # whether images should be augmented during preprocessing
augmentations = 25 # how many augmentations to make for each original image

random_seed = 7

epochs = 500
batch_size = 50
saved_models_dir = os.path.join('..', 'application', 'saved_models')

modelname = 'cnn_image_model'

In [3]:
preprocessor = ImagePreprocessor()
preprocessor.root_dir = root_dir
preprocessor.originals_dir = originals_dir
preprocessor.training_dir = training_dir
preprocessor.test_dir = test_dir
preprocessor.val_dir = val_dir
preprocessor.random_seed = random_seed
preprocessor.target_imagesize = target_imagesize
preprocessor.clear_existing_data = clear_existing_data

preprocessor.initialize()
categories = preprocessor.categories
training_count = preprocessor.training_count
validation_count = preprocessor.validation_count
test_count = preprocessor.test_count

6 image categories
465 total images

279 training images
93 validation images
93 test images

Categories:
  - animal
  - city_scape
  - food
  - group
  - landscape
  - me


In [4]:
img_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255,
    fill_mode='reflect')

train_generator = img_datagen.flow_from_directory(training_dir,
                                                   target_size=target_imagesize,
                                                   batch_size=augmentations,
                                                   shuffle=True,
                                                   seed=random_seed)

validation_generator = img_datagen.flow_from_directory(val_dir,
                                                   target_size=target_imagesize,
                                                   batch_size=augmentations,
                                                   shuffle=True,
                                                   seed=random_seed)

Found 279 images belonging to 6 classes.
Found 93 images belonging to 6 classes.


## Callback functions

After each epoch, keras will call the three functions in the section below.

- **tensorboard:** To see graphs on how our model is doing, we'll use Tensorflow's Tensorboard
- **earlystopping:** If the validation loss hasn't shown any improvement for 20 epochs in a row, it's unlikely the model will perform better with more training iterations, so we'll stop training at that point.
- **checkpointer:** Every time the validation loss improves, we save the model

In [5]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                      write_graph=True, write_images=True)

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath=os.path.join(saved_models_dir, modelname + '.hdf5'), 
                       verbose=1, save_best_only=True)

In [6]:
def predict_category(img_path, pred_model):
    img_tensor = preprocessor.file_to_tensor(img_path)
    h = pred_model.predict(img_tensor)
    return categories[np.argmax(h)]

In [7]:
def f1_score_cal(model=None):
    test_images = np.array(glob(os.path.join(test_dir, "*", "*")))
    y_true = []
    y_pred = []
    for img in test_images:
        y_true.append(categories.index(re.split(r'[\\/]',img)[-2]))
        pred = predict_category(img, model)
        y_pred.append(categories.index(pred))
    
    return f1_score(y_true, y_pred, average='weighted')

In [8]:
print("Training ", modelname)

model = InceptionV3(include_top=False, weights = 'imagenet', input_shape = (target_imagesize[0], target_imagesize[1], 3))
for layer in model.layers:
    layer.trainable = False

# custom Layers 
cus_layers = model.output
cus_layers = Flatten()(cus_layers)

cus_layers = Dense(256, activation="relu")(cus_layers)
cus_layers = Dropout(0.4)(cus_layers)
predictions = Dense(len(categories), activation="sigmoid")(cus_layers)

model_final = Model(inputs = model.input, outputs = predictions)
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])
# train the model
model_final.fit_generator(train_generator,
                         steps_per_epoch=training_count // 10, 
                          epochs=epochs,
                         validation_data = validation_generator,
                         validation_steps=validation_count // 10,
                         callbacks=[checkpointer, tensorboard, earlystopping],
                         verbose=1)

model_final.load_weights(filepath=os.path.join(saved_models_dir, modelname + '.hdf5'))

Training  cnn_image_model
Epoch 1/500
26/27 [===========================>..] - ETA: 1s - loss: 1.7847 - acc: 0.2627

C:\Users\donke\Anaconda2\envs\cnn_image_classification\lib\site-packages\PIL\Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Epoch 00000: val_loss improved from inf to 1.65101, saving model to ..\application\saved_models\cnn_image_model.hdf5
27/27 [==============================] - 47s - loss: 1.7761 - acc: 0.2675 - val_loss: 1.6510 - val_acc: 0.3365
Epoch 2/500
27/27 [==============================] - 33s - loss: 1.5521 - acc: 0.4607 - val_loss: 1.3806 - val_acc: 0.5924
Epoch 3/500
27/27 [==============================] - 32s - loss: 1.3199 - acc: 0.5914 - val_loss: 1.1306 - val_acc: 0.6682
Epoch 4/500
27/27 [==============================] - 32s - loss: 1.1349 - acc: 0.6442 - val_loss: 0.8949 - val_acc: 0.7156
Epoch 5/500
27/27 [==============================] - 32s - loss: 0.9894 - acc: 0.6862 - val_loss: 0.6461 - val_acc: 0.8152
Epoch 6/500
27/27 [==============================] - 32s - loss: 0.7591 - acc: 0.7533 - val_loss: 0.6371 - val_acc: 0.8104
Epoch 7/500
27/27 [==============================] - 33s - loss: 0.8360 - acc: 0.7471 - val_loss: 0.6722 - val_acc: 0.8057
Epoch 8/500
27/27 [===============

27/27 [==============================] - 32s - loss: 0.3246 - acc: 0.8885 - val_loss: 0.5611 - val_acc: 0.8246
Epoch 32/500
27/27 [==============================] - 31s - loss: 0.2817 - acc: 0.9046 - val_loss: 0.5407 - val_acc: 0.8199
Epoch 33/500
27/27 [==============================] - 33s - loss: 0.3178 - acc: 0.8854 - val_loss: 0.4806 - val_acc: 0.8246
Epoch 34/500
27/27 [==============================] - 34s - loss: 0.2825 - acc: 0.9138 - val_loss: 0.4691 - val_acc: 0.8436
Epoch 35/500
27/27 [==============================] - 32s - loss: 0.2610 - acc: 0.9108 - val_loss: 0.4321 - val_acc: 0.8578
Epoch 36/500
27/27 [==============================] - 34s - loss: 0.3914 - acc: 0.8807 - val_loss: 0.5122 - val_acc: 0.8246
Epoch 37/500
27/27 [==============================] - 33s - loss: 0.3232 - acc: 0.8840 - val_loss: 0.5043 - val_acc: 0.8341
Epoch 38/500
27/27 [==============================] - 32s - loss: 0.2406 - acc: 0.9287 - val_loss: 0.4983 - val_acc: 0.8294
Epoch 39/500
27/27 [=

In [9]:
print(f1_score_cal(model_final))

0.8357771261
